In [18]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [3]:
pip install -r requirements.txt

105.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 1.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 2.1 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.1
    Uninstalling python-dotenv-1.0.1:
      Successfully uninstalled python-dotenv-1.0.1
  Attempting uninstall: llamaindex-py-client
    Found existing installation: llamaindex-py-client 0.1.15
    Uninstalling llamaindex-py-client-0.1.15:
      Successfully uninstalled llamaindex-py-client-0.1.15
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.25.post1
    Uninstalling llama-index-core-0.10.25.post1:
      Successfully uninstalled llama-index-core-0.10.25.post1
  Attempting uninstall: llama-index-llms-openai
    Found existing installation: llama-index-llms-openai 0.1.13
    Uninsta

In [19]:
import nest_asyncio

nest_asyncio.apply()

In [20]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [21]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

print(nodes)

[TextNode(id_='46815d38-05d8-452c-b47b-d9f8f669c5be', embedding=None, metadata={'page_label': '1', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-12', 'last_modified_date': '2024-05-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bf5e1e20-3e79-41b7-896e-41f1ac8c3027', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-12', 'last_modified_date': '2024-05-12'}, hash='01fff8246b686cd4fe8889be21d54fde91040df44d59e9a80e663279f5193391')}, text='Preprint\nMETAGPT: M ETA

In [25]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-4-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
    


In [26]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)
    

In [27]:
summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True
    )

In [28]:
from llama_index.core.tools import QueryEngineTool

vector_query_engine = vector_index.as_query_engine()
    
summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description=(
            "Useful for summarization questions related to MetaGPT"
        ),
)
    
vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description=(
            "Useful for retrieving specific context from the MetaGPT paper."
        ),
    )

In [29]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[
            summary_tool,
            vector_tool,
        ],
        verbose=True
    )

https://docs.llamaindex.ai/en/stable/examples/query_engine/RouterQueryEngine/

In [30]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice directly mentions 'summarization', which aligns with the need to summarize the document..
The document describes MetaGPT, a meta-programming framework designed to enhance multi-agent collaborations using large language models (LLMs). MetaGPT integrates standardized operating procedures (SOPs) to streamline workflows and improve task execution by encoding these procedures into prompt sequences. This framework assigns specific roles and expertise to different agents, facilitating efficient task decomposition and execution. MetaGPT has shown superior performance in software engineering benchmarks, particularly in generating coherent solutions and handling complex software projects. The framework also incorporates an executable feedback mechanism to debug and optimize code during runtime, further enhancing its effectiveness. The document details the structure, functionalities, and experimental validations of MetaGPT, highlighting its advantages over ex